# Resumo deste notebook

### **TP1 ML - Alexandre Gomes Caldeira - 2024666544**

Abaixo apresento minha solução ao Trabalho Prático 1 da disciplina DIP DCC831 PG3 - Tópicos Especiais em Ciência da Computação - Aprendizado de Máquina. Nessa seção é dado um resumo dos métodos e resultados obtidos como resposta aos requisitos do trabalho, cuja implementação pode ser vista em detalhes nas seções seguintes.

Está disponível também um [Repositório Github](https://github.com/Alexandre-Caldeira/tp1_ml_mnist) ([github.com/Alexandre-Caldeira/tp1_ml_mnist](https://github.com/Alexandre-Caldeira/tp1_ml_mnist)) com histórico de implementação e resultados parciais. O repositório inclui requisitos para o ambiente de execução deste notebook para replicação dos resultados. Vale dizer que a execução dos resultados é demorada: 18hrs em CPU i7-8565U, 16 GB RAM.

Método: 
- Aplicada validação cruzada com 5 *folds* (StratifiedKFold) em redes neuronais com regularização por decaimento de peso ajustado de maneira automatizada
- Cada modelo teve taxa de aprendizado: 0.5, 1 e 10; neurônios na camada oculta: 25, 50 e 100; número de exemplos para cálculo de gradiente: 1 (SGD), 10 (mini-batch SGD), 50 (mini-batch SGD) e 3350 (todos os dados, GD);
- O resultado de validação cruzada dos modelos regularizados é comparado, esperando-se observar e ilustrar o ["*bias-variance trade-off*"](https://arxiv.org/abs/1812.11118).

Principais Resultados:
1. Taxa de Aprendizado menor atinge performances melhores
2. Mais exemplos para cada cálculo de gradiente resulta em performances melhores
3. Mais neurônios na camada oculta não necessariamente melhora os resultados, parece haver um limite
4. Mais neurônios na camada oculta performam melhor com taxas menores de aprendizado

Disclaimer:
- É válido aumentar o número de *folds*, variações de regularização e épocas para otimizar os resultados quantitativamente (nota final máxima de Acurácia e F1-score). Qualitativamente, nenhuma mudança será observada.

## **Pacotes utilizados**


Alguns pacotes (bibliotecas Python) foram utilizadas para:
- **Implementação da rede neuronal, descida gradiente (estocástica)** ([PyTorch](https://pytorch.org/))
- **Manipulação e visualização dos dados** ([Pandas](https://pandas.pydata.org/), [Numpy](https://numpy.org/), [Matplotlib](https://matplotlib.org/))
- **Separação de dados, implementação de métricas e validação cruzada** ([Scikit-learn](https://scikit-learn.org/))
- **Otimização automatizada do hiperparametro de regularização L2** ([Optuna](https://optuna.org/))
- **Registro de resultados e parâmetros de cada modelo durante e ao fim do treino** ([MLflow](https://mlflow.org/))

## **Sumário de resultados**

### **Método**: 

1. usando uma rede com 784 neurônios na camada de entrada (uma para pixel da imagem de entrada) e 10 neurônios na camada de saída, foram treinadas redes com 25, 50 e 100 neurônios na camada oculta com ativação sigmóide. 
2. O treino se deu por descida gradiente (clássica, estocástica e mini-batch com batches de 10 e 50 exemplos), usando como função de custo a entropia cruzada, utilizando taxa de aprendizado 0.5, 1 e 10. 
3. Foi aplicada regularização por decaimento de pesos (teoria: [Weight Decay](https://paperswithcode.com/method/weight-decay), implementação: [PyTorch Stochastic Gradient Descent with Weight Decay](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)) cujo peso $\lambda$ foi otimizado automaticamente com Optuna em 5 tentativas.
4. O treino foi conduzido empregando validação cruzada de modelos treinados independentemente em 5 amostras aleatórias diferentes do dataset original, separando 67% dos dados para treino (3350 exemplos) e 33% dos dados para teste (1650 exemplos), utilizando a estratégia de [StratifiedKFold (Scikit-learn)](https://scikit-learn.org/stable/modules/cross_validation.html#stratified-k-fold) para k = 5 com embaralhamento (shuffling). 
5. Cada treino em cada *fold* teve seus parâmetros registrados via MLflow (lr - *taxa de aprendizado*, hidden_dim - *tamanho da camada oculta*, batch_size - *número de exemplos por batch de treino* e weight decay - *decaimento de pesos*), assim como as métricas durante o treinamento (valor da função de custo em treino e teste, [accuracy e F1-score](https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9)) e fora do treinamento (valor médio de acurácia e F1-score nos *folds*, certificado por validação cruzada).


### **Principais Resultados**: Taxa de Aprendizado menor atinge performances melhores

![](imgs/lr_cv.jpg)

### **Principais Resultados**: Mais exemplos para cada cálculo de gradiente resulta em performances melhores

![](imgs/batch_size_cv.jpg)

### **Principais Resultados**: Mais neurônios na camada oculta não necessariamente melhora os resultados, parece haver um limite


![](imgs/hidden_dim_cv.jpg)


### **Principais Resultados**: Mais neurônios na camada oculta performam melhor com taxas menores de aprendizado

![](imgs/f1cv_hidden_dim_lr.jpg)

![](imgs/acccv_hidden_dim_lr.jpg)


### **Disclaimers**

Um número maior de épocas de treino e de tentativas (trials) de otimização de regularização Optuna poderiam aumentar a Acurácia e F1-score final dos modelos em geral, assim como um k maior de folds de validação cruzada poderia aumentar a confiabilidade do resultado final. Porém, devido à restrição de tempo (deadline de entrega do trabalho) optou-se por 50 épocas, 5 tentativas de otimização e 5 separações de dados para validação cruzada. Ainda assim, os resultados condizem com o esperado mediante teoria, e seguem as melhores práticas.


# Carga e Visualização dos Dados

In [ ]:
numpy.random.seed(42)